In [8]:
import re
import sys
import os
import datetime
sys.path.append('E:\Study\my_github\smart regex')

import smart_regex as sr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
patterns = [
    '姓 ?名',
    '大专|专科|自考本科|成人本科|本科|硕士|博士|博后|博士后|学士|双学士|研究生|mba',
    '(汉|壮|满|回|苗|维|维吾尔|土家|彝|蒙古|蒙|藏|布依|侗|瑶|朝鲜|白|哈尼|哈萨克|黎|傣)( *族)'
]

In [38]:
re_patterns = [re.compile(pat) for pat in patterns]
sr_patterns = [sr.compile(pat) for pat in patterns]

In [33]:
folder = r'E:\Study\my_github\smart regex\test\test_data'
lines = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            lines.extend(l.strip() for l in f.readlines())

In [34]:
%%timeit -n 1 -r 5
re_results = []
for line in lines:
    for re_pat in re_patterns:
        mat = re_pat.search(line)
        if mat:
            re_results.append(mat.group())

838 µs ± 109 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [23]:
%%timeit -n 1 -r 5
sr_results = []
for line in lines:
    for sr_pat in sr_patterns:
        mat = sr_pat.search(line)
        if mat:
            sr_results.append(mat.group())

4.18 ms ± 740 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [18]:
re.search('.', ' ')

<re.Match object; span=(0, 1), match=' '>

In [7]:
# %%timeit -n 1 -r 1
def test_sr():
#     sr_results = []
    for line in lines:
        for sr_pat in sr_patterns:
            mat = sr_pat.search(line)
#             if mat:
#                 sr_results.append(mat.group())

In [8]:
%prun test_sr()

         5664 function calls in 0.004 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      796    0.002    0.000    0.002    0.000 utils.py:103(needs_regex)
      796    0.001    0.000    0.004    0.000 data_structs.py:214(_re_func)
        1    0.000    0.000    0.004    0.004 <ipython-input-7-51d9f3d37582>:2(test_sr)
      796    0.000    0.000    0.000    0.000 data_structs.py:59(get_match_query)
     1680    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
      796    0.000    0.000    0.000    0.000 boolean.py:1142(simplify)
      796    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

# inverted index

In [9]:
def index_docs(docs=None):
    index = {}
    for doc_idx, doc in enumerate(docs):
        for line_idx, line in enumerate(doc):
            for token in sr.generate_ngram_chars(line, 2):
                if token not in index:
                    index[token] = set()
                index[token].add((doc_idx, line_idx))
    return index

In [12]:
folder = r'E:\Study\my_github\smart regex\test\test_data'
docs = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            docs.append([l.strip() for l in f.readlines()])

In [13]:
index = index_docs(docs)

In [14]:
%%timeit -n 1 -r 5
index = index_docs(docs)

12.6 ms ± 1.15 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [48]:
%%timeit -n 1 -r 5
sr_results = []

for sr_pat in sr_patterns:
    mat = sr_pat.search(docs, index)

2.61 ms ± 421 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [ ]:
处理 Match_query is True 的情况